In [57]:
#pip install tensorflow

In [58]:
#Data
import numpy as np
import pandas as pd

#Charts and visulaization
import matplotlib as mp
import matplotlib.pyplot as plt
import plotly 
import plotly.express as px
import plotly.graph_objects as go

#NNs
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [59]:
import glob
import pandas as pd

# Set the path to the folder containing the CSV files
data = r"C:\Users\Salvador\OneDrive - FCT NOVA\01- Documents\Dissertação\(R) ML_Model\Job_table_info_V2\Job_info1.xls"
data = pd.read_excel(data)

In [60]:
data.head

<bound method NDFrame.head of                jobno                                           custname  \
0      HAISID2300014    GRUPO DESPORTIVO RECREATIVO E CULTURAL PRETORIA   
1       JNBIL2301115                 SPEEDLINK PROJECTS & TRADE PVT LTD   
2      HSEPDL2300228  SEGMA SERVICOS DE ENGENHARIA, GESTAO E MANUTEN...   
3      HSEPDL2300226  ISLAND IMPORT - IMPORTACAO DE PRODUTOS ALIMENT...   
4      HSEPDL2300227  SEGMA SERVICOS DE ENGENHARIA, GESTAO E MANUTEN...   
...              ...                                                ...   
49995  HSELIS2218499              CEP II-CORREOS EXPRESS PORTUGAL, S.A.   
49996  HSELIS2218172              CEP II-CORREOS EXPRESS PORTUGAL, S.A.   
49997   AELIS2214432              CEP II-CORREOS EXPRESS PORTUGAL, S.A.   
49998   AELIS2214179              CEP II-CORREOS EXPRESS PORTUGAL, S.A.   
49999   AELIS2214254              CEP II-CORREOS EXPRESS PORTUGAL, S.A.   

          jobdate biztype  departure_loc              destino      fr

#### [Não funciona ao primeiro RUN]

In [61]:
# Assuring jobdate is in datetime format
data['jobdate'] = pd.to_datetime(data['jobdate'])


C:\Users\Salvador\Documents\Coding\Anaconda\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/04/2203' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Salvador\Documents\Coding\Anaconda\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/03/2025' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Salvador\Documents\Coding\Anaconda\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27/07/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Salvador\Documents\Coding\Anaconda\lib\site-packages\pandas\core\tools\datetimes.py:1047: User

In [62]:
data['jobdate'] = pd.to_datetime(data['jobdate'])


In [63]:
# Convert the date column to the appropriate date format
data['jobdate'] = pd.to_datetime(data['jobdate'])

# Extract the month from the date column
data['Year'] = data['jobdate'].dt.year
data['Month'] = data['jobdate'].dt.month

# Ensure that the freight cost column is in numeric format
data['frete'] = pd.to_numeric(data['frete'])

In [64]:
print(data.columns)

Index(['jobno', 'custname', 'jobdate', 'biztype', 'departure_loc', 'destino',
       'frete', 'incoterm', 'total_volume_cbm', 'ownerid', 'servicetype',
       'bizscope', 'ctnqty1', 'ctnqty2', 'ctnqty3', 'ctnqty4', 'polname',
       'podname', 'Year', 'Month'],
      dtype='object')


## DataFrame creation
#### 3D Array with the dimension of the number of enterprises

In [65]:
# Create a DataFrame with all possible combinations of Year and Month
all_months = pd.DataFrame(index=pd.MultiIndex.from_product([range(2021, 2026), range(1, 13)], names=['Year', 'Month']))

# Group your data by Year and Month
grouped_data = data.groupby(['Year', 'Month'])

# Perform the desired aggregations on your grouped data
aggregated_data = grouped_data.agg({'frete': 'sum', 'jobno': 'size'})

# Reindex the aggregated data to include all months and convert the Month column to the desired representation
aggregated_data = aggregated_data.reindex(all_months.index, fill_value=0)
aggregated_data.reset_index(inplace=True)
aggregated_data['Month'] = ((aggregated_data['Year'] - aggregated_data['Year'].min()) * 12) + aggregated_data['Month']
aggregated_data = aggregated_data[aggregated_data['Year'] <= 2025]

# Sort the data by the new representation of Month
aggregated_data.sort_values('Month', inplace=True)

# Reset the index to have a sequential order
aggregated_data.reset_index(drop=True, inplace=True)

# Rename the columns
aggregated_data.rename(columns={'frete': 'Total_Frete', 'jobno': 'Total_Jobs'}, inplace=True)

###aggregated_data


In [70]:
# Get the unique enterprises
unique_enterprises = data['custname'].unique()
num_enterprises = len(unique_enterprises)

# Determine the dimensions of the array
num_months = 60  # 5 years * 12 months
num_columns = 4

# Initialize the 3D array with zeros
array_3d = np.zeros((num_enterprises, num_months, num_columns))

# Function to fill the array with grouped data
def fill_array(group):
    enterprise_idx = np.where(unique_enterprises == group['custname'])[0][0]
    month_idx = (group['Year'] - 2021) * 12 + group['Month'] - 1
    
    # Check if the month index is within the valid range
    if month_idx < num_months:
        array_3d[enterprise_idx, month_idx, 0] = group['Year']
        array_3d[enterprise_idx, month_idx, 1] = group['Month']
        array_3d[enterprise_idx, month_idx, 2] = group['frete']
        array_3d[enterprise_idx, month_idx, 3] = group['jobno']

# Fill the array with the grouped data
grouped_data.apply(fill_array)


C:\Users\Salvador\AppData\Local\Temp\ipykernel_13588\1370491634.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  enterprise_idx = np.where(unique_enterprises == group['custname'])[0][0]


ValueError: ('Lengths must match to compare', (3557,), (2632,))

## Outlier detection


In [ ]:
plt.boxplot(data['frete'])
plt.show()

In [ ]:
from scipy.stats import zscore

# Calculate z-scores for a column
z_scores = zscore(data['frete'])

# Identify outliers based on a threshold
outlier_threshold = 3
outliers = data[abs(z_scores) > outlier_threshold]

print(outliers)
#create a list with outliers


## Tragets and Inputs Variables


In [ ]:
import pandas as pd

# Input features (X)
X = aggregated_data[['Month', 'Year', 'jobno']] 

# Target variable (y)
y = aggregated_data['frete']

X.shape

## Train-Test Split (scikit-learn library)

#### Questão: features e timestep?

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #42 assures the same split everytime

#Xtrain tem dim=2; ytrain tem dim=1
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Definir e treinar o modelo RNN 

In [ ]:
#Definir variáveis a mudar
timesteps = None
features = 4
units_RNN = 32

num_epochs = 10
batch_size = 32

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()

model.add(LSTM(units=units_RNN, input_shape=(timesteps, features), return_sequences=True)) #n sabemos quantos meses vai receber
#returnsequences faz reparar na sequencia

model.add(LSTM(units=units_RNN))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mse')
model.summary()


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))    

In [ ]:
loss = model.evaluate(X_test, y_test)